In [3]:
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")

from pynq.lib.arduino import Arduino_CAM_UART
pynq = Arduino_CAM_UART(base.ARDUINO)
import time

WRITE = 0x02
READ = 0x03
TRANSACTION = 0x04
SIMPLE = 0x05
STREAM = 0x06
STREAM_DDR = 0x07
STREAM_PIXEL = 0x08
FLOW = 0x09
CONSECUTIVE = 0x0a

In [4]:
#testing consecutive reads WITHOUT DDR. trying to use flow
#"SPOT12346789waze"
#running into issue where bytes 9-16 aren't correct
pynq.microblaze.write_blocking_command(CONSECUTIVE)
output = pynq.microblaze.read_mailbox(0,16)
print(output)
for val in output:
    print(chr(val))

[83, 80, 79, 84, 49, 50, 51, 52, 83, 80, 79, 84, 49, 50, 51, 52]
S
P
O
T
1
2
3
4
S
P
O
T
1
2
3
4


In [15]:
#testing flow control
file_size = 14400
buf0 = pynq.buf_manager.cma_alloc(file_size, data_type="uint8_t")
buf1 = pynq.buf_manager.cma_get_buffer(buf0, file_size)
phy_addr = pynq.buf_manager.cma_get_phy_addr(buf0)

for index in range(0,file_size): #check to make sure buffer is "emptied"
    buf1[index] = bytes([0])
#     print(buf1[index])

pynq.microblaze.write_mailbox(0, phy_addr)
pynq.microblaze.write_mailbox(4, file_size)
pynq.microblaze.write_blocking_command(FLOW)
#converting buf1 to list
image_vals = []
for index in range(0, file_size):
    image_vals.append(buf1[index].hex())
#     print(image_vals[index])
#     image_vals[index] = "0x" + image_vals[index]
    image_vals[-1] = int(image_vals[-1], 16)
#     image_vals[index] = ord(image_vals[index])
    
print(image_vals)
    
#convert list to list of tuples
tuples_list = []
for index in range(0, file_size, 3):
    temp = []
    for i in range(0,3):
        temp.append(image_vals[index + i])
    tuples_list.append(tuple(temp))
    
# print(tuples_list)
    
#creating image using PIL
size = (80,60)
img = Image.new("RGB", size)
img.putdata(tuples_list)

# data = numpy.asarray(img)
# print(data)

img.save('snapchat.jpg')

[84, 49, 50, 51, 52, 83, 80, 79, 79, 84, 49, 50, 51, 52, 83, 80, 52, 83, 80, 52, 83, 80, 52, 83, 80, 52, 83, 83, 52, 83, 83, 52, 83, 83, 52, 83, 83, 52, 83, 83, 52, 83, 83, 52, 83, 83, 52, 83, 83, 52, 83, 83, 52, 83, 83, 52, 83, 80, 52, 83, 80, 52, 83, 80, 52, 83, 80, 52, 83, 80, 52, 83, 80, 52, 83, 80, 52, 83, 80, 52, 83, 80, 52, 83, 79, 52, 83, 79, 52, 83, 79, 52, 83, 79, 52, 83, 79, 52, 83, 79, 52, 83, 79, 52, 83, 79, 52, 83, 79, 52, 83, 79, 52, 83, 84, 52, 83, 84, 52, 83, 84, 52, 83, 84, 52, 83, 84, 52, 83, 84, 52, 83, 84, 52, 83, 84, 52, 83, 84, 52, 83, 84, 52, 83, 84, 52, 83, 84, 52, 83, 49, 52, 83, 49, 52, 83, 49, 52, 83, 49, 52, 83, 49, 52, 83, 49, 52, 83, 49, 52, 83, 49, 52, 83, 49, 52, 83, 50, 52, 83, 50, 52, 83, 50, 52, 83, 50, 52, 83, 50, 52, 83, 50, 52, 83, 50, 52, 83, 50, 52, 83, 50, 52, 83, 50, 52, 83, 51, 52, 83, 51, 52, 83, 51, 52, 83, 51, 52, 83, 51, 52, 83, 51, 52, 83, 51, 52, 83, 51, 52, 83, 51, 52, 83, 51, 52, 83, 52, 52, 83, 52, 52, 83, 52, 52, 83, 52, 52, 83, 52,

NameError: name 'Image' is not defined

In [3]:
#testing image sending at 115200 QQQVGA 80x60 RGB565 - 14400 bytes (RGB tuples)
from PIL import Image

file_size = 14400
buf0 = pynq.buf_manager.cma_alloc(file_size, data_type="uint8_t")
buf1 = pynq.buf_manager.cma_get_buffer(buf0, file_size)
phy_addr = pynq.buf_manager.cma_get_phy_addr(buf0)

for index in range(0,file_size): #check to make sure buffer is "emptied"
    buf1[index] = bytes([0])
#     print(buf1[index])

pynq.microblaze.write_mailbox(0, phy_addr)
pynq.microblaze.write_mailbox(4, file_size)
pynq.microblaze.write_blocking_command(STREAM_PIXEL)
# read_in = pynq.microblaze.read_mailbox(0,1)
# print (read_in)

#converting buf1 to list
image_vals = []
for index in range(0, file_size):
    image_vals.append(buf1[index].hex())
#     print(image_vals[index])
#     image_vals[index] = "0x" + image_vals[index]
    image_vals[-1] = int(image_vals[-1], 16)
#     image_vals[index] = ord(image_vals[index])
    
print(image_vals)
    
#convert list to list of tuples
tuples_list = []
for index in range(0, file_size, 3):
    temp = []
    for i in range(0,3):
        temp.append(image_vals[index + i])
    tuples_list.append(tuple(temp))
    
# print(tuples_list)
    
#creating image using PIL
size = (80,60)
img = Image.new("RGB", size)
img.putdata(tuples_list)

# data = numpy.asarray(img)
# print(data)

img.save('snapchat.jpg')

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [7]:
#testing multiple consecutive uart_reads but to DDR at 9600 & 115200 baud (works for 5376 bytes)
file_size = 14400
buf0 = pynq.buf_manager.cma_alloc(file_size, data_type="uint8_t")
buf1 = pynq.buf_manager.cma_get_buffer(buf0, file_size)
phy_addr = pynq.buf_manager.cma_get_phy_addr(buf0)

for index in range(0,file_size): #check to make sure buffer is "emptied"
    buf1[index] = bytes([1])
#     print(buf1[index])

pynq.microblaze.write_mailbox(0, phy_addr)
pynq.microblaze.write_mailbox(4, file_size)
pynq.microblaze.write_blocking_command(STREAM_DDR)
read_in = pynq.microblaze.read_mailbox(0,128)

for index in range(0,file_size):
    print(buf1[index].decode("utf-8") + " " + str(index))
#     print(buf1[index])

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte

In [13]:
#testing multiple consecutive uart_reads but to DDR at 9600 & 115200 baud (works for 5376 bytes)
file_size = 5376
buf0 = pynq.buf_manager.cma_alloc(file_size, data_type="uint8_t")
buf1 = pynq.buf_manager.cma_get_buffer(buf0, file_size)
phy_addr = pynq.buf_manager.cma_get_phy_addr(buf0)

for index in range(0,file_size): #check to make sure buffer is "emptied"
    buf1[index] = bytes([0])
#     print(buf1[index])

pynq.microblaze.write_mailbox(0, phy_addr)
pynq.microblaze.write_mailbox(4, file_size)
pynq.microblaze.write_blocking_command(STREAM_DDR)
read_in = pynq.microblaze.read_mailbox(0,128)

for index in range(0,file_size):
    print(buf1[index].decode("utf-8") + " " + str(index))

P 0
P 1
O 2
T 3
S 4
P 5
O 6
T 7
S 8
P 9
O 10
T 11
S 12
P 13
O 14
T 15
P 16
P 17
O 18
T 19
S 20
P 21
O 22
T 23
S 24
P 25
O 26
T 27
S 28
P 29
O 30
T 31
P 32
P 33
O 34
T 35
S 36
P 37
O 38
T 39
S 40
P 41
O 42
T 43
S 44
P 45
O 46
T 47
P 48
P 49
O 50
T 51
S 52
P 53
O 54
T 55
S 56
P 57
O 58
T 59
S 60
P 61
O 62
T 63
P 64
P 65
O 66
T 67
S 68
P 69
O 70
T 71
S 72
P 73
O 74
T 75
S 76
P 77
O 78
T 79
P 80
P 81
O 82
T 83
S 84
P 85
O 86
T 87
S 88
P 89
O 90
T 91
S 92
P 93
O 94
T 95
P 96
P 97
O 98
T 99
S 100
P 101
O 102
T 103
S 104
P 105
O 106
T 107
S 108
P 109
O 110
T 111
P 112
P 113
O 114
T 115
S 116
P 117
O 118
T 119
S 120
P 121
O 122
T 123
S 124
P 125
O 126
T 127
P 128
P 129
O 130
T 131
S 132
P 133
O 134
T 135
S 136
P 137
O 138
T 139
S 140
P 141
O 142
T 143
P 144
P 145
O 146
T 147
S 148
P 149
O 150
T 151
S 152
P 153
O 154
T 155
S 156
P 157
O 158
T 159
P 160
P 161
O 162
T 163
S 164
P 165
O 166
T 167
S 168
P 169
O 170
T 171
S 172
P 173
O 174
T 175
P 176
P 177
O 178
T 179
S 180
P 181
O 182
T 183
S 184


P 1465
O 1466
T 1467
S 1468
P 1469
O 1470
T 1471
P 1472
P 1473
O 1474
T 1475
S 1476
P 1477
O 1478
T 1479
S 1480
P 1481
O 1482
T 1483
S 1484
P 1485
O 1486
T 1487
P 1488
P 1489
O 1490
T 1491
S 1492
P 1493
O 1494
T 1495
S 1496
P 1497
O 1498
T 1499
S 1500
P 1501
O 1502
T 1503
P 1504
P 1505
O 1506
T 1507
S 1508
P 1509
O 1510
T 1511
S 1512
P 1513
O 1514
T 1515
S 1516
P 1517
O 1518
T 1519
P 1520
P 1521
O 1522
T 1523
S 1524
P 1525
O 1526
T 1527
S 1528
P 1529
O 1530
T 1531
S 1532
P 1533
O 1534
T 1535
P 1536
P 1537
O 1538
T 1539
S 1540
P 1541
O 1542
T 1543
S 1544
P 1545
O 1546
T 1547
S 1548
P 1549
O 1550
T 1551
P 1552
P 1553
O 1554
T 1555
S 1556
P 1557
O 1558
T 1559
S 1560
P 1561
O 1562
T 1563
S 1564
P 1565
O 1566
T 1567
P 1568
P 1569
O 1570
T 1571
S 1572
P 1573
O 1574
T 1575
S 1576
P 1577
O 1578
T 1579
S 1580
P 1581
O 1582
T 1583
P 1584
P 1585
O 1586
T 1587
S 1588
P 1589
O 1590
T 1591
S 1592
P 1593
O 1594
T 1595
S 1596
P 1597
O 1598
T 1599
P 1600
P 1601
O 1602
T 1603
S 1604
P 1605
O 1606
T 1607

P 2656
P 2657
O 2658
T 2659
S 2660
P 2661
O 2662
T 2663
S 2664
P 2665
O 2666
T 2667
S 2668
P 2669
O 2670
T 2671
P 2672
P 2673
O 2674
T 2675
S 2676
P 2677
O 2678
T 2679
S 2680
P 2681
O 2682
T 2683
S 2684
P 2685
O 2686
T 2687
P 2688
P 2689
O 2690
T 2691
S 2692
P 2693
O 2694
T 2695
S 2696
P 2697
O 2698
T 2699
S 2700
P 2701
O 2702
T 2703
P 2704
P 2705
O 2706
T 2707
S 2708
P 2709
O 2710
T 2711
S 2712
P 2713
O 2714
T 2715
S 2716
P 2717
O 2718
T 2719
P 2720
P 2721
O 2722
T 2723
S 2724
P 2725
O 2726
T 2727
S 2728
P 2729
O 2730
T 2731
S 2732
P 2733
O 2734
T 2735
P 2736
P 2737
O 2738
T 2739
S 2740
P 2741
O 2742
T 2743
S 2744
P 2745
O 2746
T 2747
S 2748
P 2749
O 2750
T 2751
P 2752
P 2753
O 2754
T 2755
S 2756
P 2757
O 2758
T 2759
S 2760
P 2761
O 2762
T 2763
S 2764
P 2765
O 2766
T 2767
P 2768
P 2769
O 2770
T 2771
S 2772
P 2773
O 2774
T 2775
S 2776
P 2777
O 2778
T 2779
S 2780
P 2781
O 2782
T 2783
P 2784
P 2785
O 2786
T 2787
S 2788
P 2789
O 2790
T 2791
S 2792
P 2793
O 2794
T 2795
S 2796
P 2797
O 2798

O 3970
T 3971
S 3972
P 3973
O 3974
T 3975
S 3976
P 3977
O 3978
T 3979
S 3980
P 3981
O 3982
T 3983
P 3984
P 3985
O 3986
T 3987
S 3988
P 3989
O 3990
T 3991
S 3992
P 3993
O 3994
T 3995
S 3996
P 3997
O 3998
T 3999
P 4000
P 4001
O 4002
T 4003
S 4004
P 4005
O 4006
T 4007
S 4008
P 4009
O 4010
T 4011
S 4012
P 4013
O 4014
T 4015
P 4016
P 4017
O 4018
T 4019
S 4020
P 4021
O 4022
T 4023
S 4024
P 4025
O 4026
T 4027
S 4028
P 4029
O 4030
T 4031
P 4032
P 4033
O 4034
T 4035
S 4036
P 4037
O 4038
T 4039
S 4040
P 4041
O 4042
T 4043
S 4044
P 4045
O 4046
T 4047
P 4048
P 4049
O 4050
T 4051
S 4052
P 4053
O 4054
T 4055
S 4056
P 4057
O 4058
T 4059
S 4060
P 4061
O 4062
T 4063
P 4064
P 4065
O 4066
T 4067
S 4068
P 4069
O 4070
T 4071
S 4072
P 4073
O 4074
T 4075
S 4076
P 4077
O 4078
T 4079
P 4080
P 4081
O 4082
T 4083
S 4084
P 4085
O 4086
T 4087
S 4088
P 4089
O 4090
T 4091
S 4092
P 4093
O 4094
T 4095
P 4096
P 4097
O 4098
T 4099
S 4100
P 4101
O 4102
T 4103
S 4104
P 4105
O 4106
T 4107
S 4108
P 4109
O 4110
T 4111
P 4112

S 5276
P 5277
O 5278
T 5279
P 5280
P 5281
O 5282
T 5283
S 5284
P 5285
O 5286
T 5287
S 5288
P 5289
O 5290
T 5291
S 5292
P 5293
O 5294
T 5295
P 5296
P 5297
O 5298
T 5299
S 5300
P 5301
O 5302
T 5303
S 5304
P 5305
O 5306
T 5307
S 5308
P 5309
O 5310
T 5311
P 5312
P 5313
O 5314
T 5315
S 5316
P 5317
O 5318
T 5319
S 5320
P 5321
O 5322
T 5323
S 5324
P 5325
O 5326
T 5327
P 5328
P 5329
O 5330
T 5331
S 5332
P 5333
O 5334
T 5335
S 5336
P 5337
O 5338
T 5339
S 5340
P 5341
O 5342
T 5343
P 5344
P 5345
O 5346
T 5347
S 5348
P 5349
O 5350
T 5351
S 5352
P 5353
O 5354
T 5355
S 5356
P 5357
O 5358
T 5359
P 5360
P 5361
O 5362
T 5363
S 5364
P 5365
O 5366
T 5367
S 5368
P 5369
O 5370
T 5371
S 5372
P 5373
O 5374
T 5375


In [ ]:
#need to test with camera snapshot between recv and send on camera side. make sure they sync up
#may need to throw in an acknowledge signal

In [4]:
#testing calling multiple consecutive uart_reads (128 bytes in 16 byte read chunks)
#only test working with 9600 baud so far and not writing to DDR

pynq.microblaze.write_blocking_command(STREAM)
read_in = pynq.microblaze.read_mailbox(0,128)
# for index,val in enumerate(read_in):
#     if(index == 0):
#         print("Num Read In: " + str(val))
#     else:
#         print(chr(val))

for index,val in enumerate(read_in):
    print(chr(val) + " " + str(index))

# counter = 0;
# while counter < 128:
#     pynq.microblaze.write_mailbox(counter, 0)
#     counter += 4
#     print(counter)

OverflowError: Python int too large to convert to C long

In [9]:
pynq.microblaze.write_blocking_command(SIMPLE)
print(pynq.microblaze.read_mailbox(0,5))
read_in = pynq.microblaze.read_mailbox(0,5)
for index,val in enumerate(read_in):
    if(index != 0):
        print(chr(val))
pynq.microblaze.write_mailbox(0, 0)
pynq.microblaze.write_mailbox(4, 0)

[4, 83, 80, 79, 84]
S
P
O
T


In [34]:
#Transaction Code - sending snapshot command and saving image to ddr
from PIL import Image
from numpy import array

file_size = 4800 #edit this to file size. num of bytes. QQQQVGA = 4800, VGA = 921600
buf0 = pynq.buf_manager.cma_alloc(file_size, data_type="uint8_t")
buf1 = pynq.buf_manager.cma_get_buffer(buf0, file_size)
phy_addr = pynq.buf_manager.cma_get_phy_addr(buf0)

f = open("snapshot.jpg", "w")

pynq.microblaze.write_mailbox(0, phy_addr)
pynq.microblaze.write_mailbox(4, file_size)
pynq.microblaze.write_blocking_command(TRANSACTION)
print(pynq.microblaze.read_mailbox(0,1))

for index in range (0, file_size):
    f.write(buf1[index].decode("utf-8"))

f.close()

4800


In [2]:
pynq.microblaze.write_blocking_command(WRITE)
print(pynq.microblaze.read_mailbox(0,1))

69


In [8]:
pynq.microblaze.write_blocking_command(READ)
read_in = pynq.microblaze.read_mailbox(0,4)
print(read_in)
for val in read_in:
    print(chr(val))


[83, 80, 79, 84]
S
P
O
T
